In [ ]:
import os
import pprint

In [ ]:
import open3d.ml as _ml3d
import open3d.ml.tf as ml3d
from open3d.ml.datasets import (SemanticKITTI, ParisLille3D, Semantic3D, S3DIS, Toronto3D, KITTI)
from open3d.ml.tf.models import RandLANet, KPFCNN
from open3d.ml.tf.pipelines import SemanticSegmentation

In [ ]:
dataset_path='/home/threedee/datasets/S3DIS'
randlanet_s3dis_cfg = "/home/threedee/repos/Open3D-ML/ml3d/configs/randlanet_s3dis.yml"

In [ ]:
cfg = _ml3d.utils.Config.load_from_file(randlanet_s3dis_cfg)
cfg.dataset['dataset_path'] = dataset_path

model = RandLANet(**cfg.model)
dataset = S3DIS(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [ ]:
# construct a dataset by specifying dataset_path
#dataset = ml3d.datasets.S3DIS(dataset_path=dataset_path)

# get the 'all' split that combines training, validation and test set
all_split = dataset.get_split('all')

# print the attributes of the first datum
print(all_split.get_attr(0))

# print the shape of the first point cloud
print(all_split.get_data(0)['point'].shape)

# show the first 100 frames using the visualizer
vis = ml3d.vis.Visualizer()

vis.visualize_dataset(dataset, 'all', indices=range(100))

In [ ]:
# download the weights.
ckpt_path = "/home/threedee/repos/Open3D-ML/logs/RandLANet_S3DIS_tf/checkpoint/ckpt-6"

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

In [ ]:
#odel = RandLANet(**cfg.model)
#dataset = S3DIS(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
#pipeline = SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

pipeline.cfg_tb = {
    'readme': "readme",
    'cmd_line': "cmd_line",
    'dataset': pprint.pformat("S3DIS", indent=2),
    'model': pprint.pformat("RandLaNet", indent=2),
    'pipeline': pprint.pformat("SemanticSegmentation", indent=2)
}

# prints training progress in the console.
pipeline.run_train()

In [ ]:
import numpy as np
import time

start=time.time()

data = np.load('/home/threedee/repos/Open3D-ML/logs/cache/e08af6b877145892abd30193e3fa5056/Area_1_copyRoom_1.npy', allow_pickle=True)

end=time.time()

print("\nData summary:\n", data)
print("\nData shape:\n", data.shape)
print(f"\nTime to read: {round(end-start,5)} seconds.")



In [ ]:
data.item()
